<a href="https://colab.research.google.com/github/Piyumi22/UCSD-Microgrid-Database/blob/master/Wind_EDA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>